In [ ]:
# !pip install torchmetrics


In [ ]:
import os, sys, glob, argparse
import pandas as pd
import numpy as np
# from tqdm import tqdm

import cv2, time
# from PIL import Image
# from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchmetrics
import torchvision.models as models
# import torchvision.transforms as transforms
# import torchvision.datasets as datasets
import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.autograd import Variable
from torch.utils.data.dataset import Dataset


# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
!git clone https://github.com/Hpeox/AI_Camp.git

Cloning into 'AI_Camp'...
remote: Enumerating objects: 29073, done.
remote: Counting objects: 100% (1899/1899), done.
remote: Compressing objects: 100% (1793/1793), done.
remote: Total 29073 (delta 100), reused 1895 (delta 97), pack-reused 27174
Receiving objects: 100% (29073/29073), 237.79 MiB | 16.70 MiB/s, done.
Resolving deltas: 100% (101/101), done.
Updating files: 100% (30609/30609), done.


In [ ]:
!ls


AI_Camp  sample_data


In [ ]:
# 读取数据集
train_path = glob.glob("/content/AI_Camp/CNN/data/train/*")
test_path = glob.glob("/content/AI_Camp/CNN/data/test/*")

train_path.sort()
test_path.sort()

train_df = pd.read_csv("/content/AI_Camp/CNN/data/train.csv")
train_df = train_df.sort_values(by="name")
train_label = train_df["label"].values

# 自定义数据集
# 带有图片缓存的逻辑
DATA_CACHE = {}


class XunFeiDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None

    def __getitem__(self, index):
        if self.img_path[index] in DATA_CACHE:
            img = DATA_CACHE[self.img_path[index]]
        else:
            img = cv2.imread(self.img_path[index])
            DATA_CACHE[self.img_path[index]] = img
        if self.transform is not None:
            img = self.transform(image=img)["image"]
        img = img.transpose([2, 0, 1])
        return img, torch.from_numpy(np.array(self.img_label[index]))

    def __len__(self):
        return len(self.img_path)


import albumentations as A

dataset = XunFeiDataset(
    train_path,
    train_label,
    A.Compose(
        [
            A.RandomRotate90(),
            A.Resize(256, 256),
            A.RandomCrop(224, 224),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ]
    ),
)
train_dataset, test_dataset = torch.utils.data.random_split(
    dataset=dataset, lengths=[0.95, 0.05], generator=torch.Generator().manual_seed(42)
)

# 训练集
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, shuffle=True, num_workers=0, pin_memory=False
)

# 验证集
val_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=False
)

# 测试集
test_loader = torch.utils.data.DataLoader(
    XunFeiDataset(
        test_path,
        [0] * len(test_path),
        A.Compose(
            [
                A.Resize(256, 256),
                A.RandomCrop(224, 224),
                A.HorizontalFlip(p=0.5),
                # A.RandomBrightnessContrast(p=0.5),
                A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ]
        ),
    ),
    batch_size=50,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
)

In [ ]:
class XunFeiNet50(nn.Module):
    def __init__(self):
        super(XunFeiNet50, self).__init__()
        model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, 25)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out

class XunFeiNet(nn.Module):
    def __init__(self):
        super(XunFeiNet, self).__init__()
        model = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, 25)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out
class XunFeiNet152(nn.Module):
    def __init__(self):
        super(XunFeiNet152, self).__init__()
        model = models.resnet152(weights=models.ResNet152_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, 25)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out
model50 = XunFeiNet50().to(device)
model101= XunFeiNet().to(device)
model152= XunFeiNet152().to(device)
# model = model.to(device)
criterion = nn.CrossEntropyLoss().cuda()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 83.8MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to /root/.cache/torch/hub/checkpoints/resnet101-cd907fc2.pth
100%|██████████| 171M/171M [00:00<00:00, 221MB/s]
Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to /root/.cache/torch/hub/checkpoints/resnet152-f82ba261.pth
100%|██████████| 230M/230M [00:04<00:00, 56.0MB/s]


In [ ]:
# 模型训练
def train(train_loader, model, criterion, optimizer):
    start = time.time()
    start_batch = [start, 0]
    model.train()
    train_loss = 0.0
    preds = torch.tensor([])
    target_all = torch.tensor([])
    for i, (input, target) in enumerate(train_loader):
        input = input.to(device)
        target = target.to(device)
        output = model(input)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            start_batch[((i + 1) // 100) % 2] = time.time()
            print(
                "Train loss",
                loss.item(),
                "t={}s".format(
                    start_batch[((i + 1) // 100) % 2]
                    - start_batch[(1 + (i + 1) // 100) % 2]
                ),
            )

        preds = torch.cat((preds, output.cpu().argmax(1)))
        target_all = torch.cat((target_all, target.cpu()))

        train_loss += loss.item()

    val_acc = torchmetrics.functional.classification.multiclass_f1_score(
        preds, target_all, num_classes=25, average="macro"
    )
    print("t={}s".format(time.time() - start))
    print("F1 score", val_acc)
    return train_loss / len(train_loader)


# 模型验证
def validate(val_loader, model, criterion):
    model.eval()
    val_acc = 0.0
    preds = torch.tensor([])
    target_all = torch.tensor([])
    val_loss=0.0
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            input = input.to(device)
            target = target.to(device)
            output = model(input)
            loss = criterion(output, target)
            # val_acc += (output.argmax(1) == target).sum().item()
            preds = torch.cat((preds, output.cpu().argmax(1)))
            target_all = torch.cat((target_all, target.cpu()))

        val_acc = torchmetrics.functional.classification.multiclass_f1_score(
            preds, target_all, num_classes=25, average="macro"
        )
        val_loss += loss.item()
    # return val_acc / len(val_loader.dataset)
    return val_acc,val_loss/len(val_loader)


# 模型预测
def predict(test_loader, model):
    model.eval()

    test_pred = []
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input = input.to(device)
            output = model(input)
            test_pred.append(output.data.cpu().numpy())

    return np.vstack(test_pred)

In [ ]:
# for best model
model=model152
optimizer = torch.optim.AdamW(model.parameters(), 0.0001)
# model=model101
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"/content/model/baseline_152.pth")

Train loss 3.0476365089416504 t=88.26209783554077s
Train loss 2.0708141326904297 t=77.12735915184021s
Train loss 1.7472223043441772 t=77.31742024421692s
Train loss 1.7517409324645996 t=76.69264245033264s
Train loss 1.0861563682556152 t=76.21940469741821s
Train loss 1.1610586643218994 t=75.78902220726013s
t=534.5744822025299s
F1 score tensor(0.3917)
epoch 1 : train_loss: 1.9450501847722281 val_loss: 0.03508191969659594 val_f1: tensor(0.6550)
Train loss 1.1151342391967773 t=74.02593159675598s
Train loss 0.7412101626396179 t=74.27640891075134s
Train loss 0.8763712644577026 t=74.3703544139862s
Train loss 0.9524921774864197 t=74.46224045753479s
Train loss 0.6785811185836792 t=74.34841799736023s
Train loss 0.9604212045669556 t=74.33931946754456s
t=505.7985739707947s
F1 score tensor(0.7083)
epoch 2 : train_loss: 0.8310266115241814 val_loss: 0.02287891341580285 val_f1: tensor(0.7724)
Train loss 0.3797929286956787 t=74.01039528846741s
Train loss 0.5162652134895325 t=74.4267566204071s
Train loss

In [ ]:
# for best model
model=model101
optimizer = torch.optim.AdamW(model.parameters(), 0.0001)
# model=model101
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"./model/baseline_101.pth")

Train loss 1.9190107583999634 t=55.2235164642334s
Train loss 1.604310154914856 t=54.169617891311646s
Train loss 1.5684213638305664 t=54.4313588142395s
Train loss 1.1803944110870361 t=54.01967525482178s
Train loss 1.21957528591156 t=54.55041790008545s
Train loss 0.7174181342124939 t=54.105995416641235s
t=370.2995150089264s
F1 score tensor(0.4827)
epoch 1 : train_loss: 1.5851940297345233 val_loss: 0.020687128106753033 val_f1: tensor(0.6197)
Train loss 0.7935527563095093 t=53.69859838485718s
Train loss 0.5381841659545898 t=54.11372089385986s
Train loss 0.8454071283340454 t=54.05889296531677s
Train loss 0.7152472138404846 t=53.9410765171051s
Train loss 0.40476733446121216 t=54.28064298629761s
Train loss 0.5825835466384888 t=54.00325417518616s
t=367.86373949050903s
F1 score tensor(0.7225)
epoch 2 : train_loss: 0.801320823716347 val_loss: 0.019692533546023898 val_f1: tensor(0.7765)
Train loss 0.4299902617931366 t=53.609546184539795s
Train loss 0.6208288073539734 t=54.09341382980347s
Train lo

In [ ]:
# for best model
model=model50
optimizer = torch.optim.AdamW(model.parameters(), 0.0001)
# model=model101
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"./model/baseline_50.pth")

Train loss 3.0778005123138428 t=36.45119881629944s
Train loss 2.9839706420898438 t=34.90014696121216s
Train loss 2.4677653312683105 t=34.40766882896423s
Train loss 1.9801123142242432 t=35.00399112701416s
Train loss 1.7957130670547485 t=35.91481256484985s
Train loss 1.6115543842315674 t=35.30828809738159s
t=240.06659722328186s
F1 score tensor(0.2579)
epoch 1 : train_loss: 2.3984916362468365 val_loss: 0.03880749146143595 val_f1: tensor(0.4967)
Train loss 1.3293904066085815 t=34.67344927787781s
Train loss 1.2297985553741455 t=34.67178559303284s
Train loss 1.791744351387024 t=34.863085985183716s
Train loss 0.9549806118011475 t=34.56373620033264s
Train loss 1.0414737462997437 t=34.538920402526855s
Train loss 0.9631244540214539 t=34.403910636901855s
t=235.643319606781s
F1 score tensor(0.6050)
epoch 2 : train_loss: 1.1759235292629349 val_loss: 0.020151206188731723 val_f1: tensor(0.6879)
Train loss 0.9067736864089966 t=34.287184953689575s
Train loss 1.1491647958755493 t=34.702731132507324s
Tra

In [ ]:
class XunFeiNet34(nn.Module):
    def __init__(self):
        super(XunFeiNet34, self).__init__()
        model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(512, 25)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out
model34 = XunFeiNet34().to(device)

# for best model
model=model34
optimizer = torch.optim.AdamW(model.parameters(), 0.0001)
# model=model101
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"./model/baseline_34.pth")

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 61.5MB/s]


Train loss 2.675316095352173 t=21.672868013381958s
Train loss 1.9864163398742676 t=20.929474592208862s
Train loss 2.0389387607574463 t=19.96168303489685s
Train loss 1.6547023057937622 t=20.795820713043213s
Train loss 1.5560764074325562 t=20.140869140625s
Train loss 1.2593016624450684 t=20.451364755630493s
t=140.70986008644104s
F1 score tensor(0.4035)
epoch 1 : train_loss: 1.8789681092933062 val_loss: 0.02416436539755927 val_f1: tensor(0.5753)
Train loss 1.5519540309906006 t=20.127243280410767s
Train loss 1.2608009576797485 t=20.482006788253784s
Train loss 0.9310277104377747 t=20.065386056900024s
Train loss 0.8443719148635864 t=20.39361023902893s
Train loss 1.356546401977539 t=20.442185163497925s
Train loss 0.8738254308700562 t=20.57886028289795s
t=138.82112073898315s
F1 score tensor(0.6479)
epoch 2 : train_loss: 1.045486842877014 val_loss: 0.024244331651263766 val_f1: tensor(0.6599)
Train loss 0.6981716156005859 t=20.582112550735474s
Train loss 0.7801882028579712 t=20.508065938949585s


In [ ]:
# 对测试集多次预测
pred = None
model=model101
model.load_state_dict(torch.load("./model/baseline_101.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
model=model152
model.load_state_dict(torch.load("./model/baseline_152.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
model=model50
model.load_state_dict(torch.load("./model/baseline_50.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += 0.8*predict(test_loader, model)
    print(_+1)
model=model34
model.load_state_dict(torch.load("./model/baseline_34.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += 0.8*predict(test_loader, model)
    print(_+1)
submit = pd.DataFrame(
    {
        'name': [x.split('/')[-1] for x in test_path],
        'label': pred.argmax(1)
})

# 生成提交结果
submit = submit.sort_values(by='name')
submit.to_csv('submit7.csv', index=None)

1
2
3
1
2
3
1
2
3
1
2
3


In [ ]:
model.train()
train_loss = 0.0
model = model.to(device)
for i, (input, target) in enumerate(train_loader):
    input = input.to(device)
    target = target.to(device)
    output = model(input)
    loss = criterion(output, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
epochs=4
model=model101
optimizer = torch.optim.AdamW(model.parameters(), 0.0001)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
torch.save(model.state_dict(),"./model/baseline_test.pth")



In [ ]:
epochs=3
model101= XunFeiNet().to(device)
model=model101
optimizer = torch.optim.AdamW(model.parameters(), 0.0001)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
torch.save(model.state_dict(),"./model/baseline_101.pth")

In [ ]:
pred = None
model=model101
model.load_state_dict(torch.load("./model/baseline_test.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
model.load_state_dict(torch.load("./model/baseline_101.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
model=model50
model.load_state_dict(torch.load("./model/baseline_50.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
submit = pd.DataFrame(
    {
        'name': [x.split('/')[-1] for x in test_path],
        'label': pred.argmax(1)
})

# 生成提交结果
submit = submit.sort_values(by='name')
submit.to_csv('submit6.csv', index=None)

In [ ]:
model = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
model.fc

In [ ]:
val_acc = validate(val_loader, model)
val_acc

In [ ]:
dataset=XunFeiDataset(train_path[:-1000], train_label[:-1000],
            A.Compose([
            # A.RandomRotate90(),
            A.Resize(256, 256),
            A.RandomCrop(224, 224),
            # A.HorizontalFlip(p=0.5),
            # A.RandomBrightnessContrast(p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        )
train_dataset, test_dataset=torch.utils.data.random_split(dataset=dataset,lengths=[0.9,0.1],generator=torch.Generator().manual_seed(42))
train_dataset

In [ ]:
model.load_state_dict(torch.load("./model/baseline_test.pth"))
val_acc,val_loss = validate(val_loader, model, criterion)
print(val_loss)
len(val_loader.dataset)